<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/FCN_Experiments/MNIST_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/FCN_Experiments/'

In [ ]:
import BayesKeras
import BayesKeras.optimizers as optimizers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#tf.debugging.set_log_device_placement(True)
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 0 --opt HMC --gpu 0 &
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 2 --opt HMC --gpu 1 &

dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'HMC', 'gpu': '0'}

eps = dict['eps']
lam = dict['lam']
rob = dict['rob']
optim = dict['opt']
gpu = dict['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

#X_train = X_train[0:10000]
#y_train = y_train[0:10000]

model = Sequential()
model.add(Dense(512, activation="relu", input_shape=(None, None, 28*28)))
model.add(Dense(10, activation="softmax"))

inf = 2
full_covar = False
if(optim == 'VOGN'):
    # was 0.25 for a bit
    inf = 2
    learning_rate = 0.35; decay=0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif(optim == 'BBB'):
    inf = 10
    learning_rate = 0.45; decay=0.0
    opt = optimizers.BayesByBackprop()
elif(optim == 'SWAG'):
    learning_rate = 0.01; decay=0.0
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SWAG-FC'):
    learning_rate = 0.01; decay=0.0; full_covar=True
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SGD'):
    learning_rate = 1.0; decay=0.0
    opt = optimizers.StochasticGradientDescent()
elif(optim == 'NA'):
    inf = 2
    learning_rate = 0.001; decay=0.0
    opt = optimizers.NoisyAdam()
elif(optim == 'ADAM'):
    learning_rate = 0.00001; decay=0.0
    opt = optimizers.Adam()
elif(optim == 'HMC'):
    learning_rate = 0.075; decay=0.0; inf=250
    linear_schedule = False
    opt = optimizers.HamiltonianMonteCarlo()

# Compile the model to train with Bayesian inference
if(rob == 0):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif(rob != 0):
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

bayes_model = opt.compile(model, loss_fn=loss, epochs=20, learning_rate=learning_rate,
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, inflate_prior=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam) #, preload="SGD_FCN_Posterior_1")



This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
BayesKeras: Using implicit prior
(784, 512) 0.5646924393157821
(512, 10) 0.6987712429686843
BayesKeras: Using implicit prior
(784, 512) 0.5646924393157821
(512, 10) 0.6987712429686843
<class 'list'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
[<tf.Tensor: shape=(784, 512), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(512,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 

In [ ]:
#steps was 50
# Train the model on your data
bayes_model.train(X_train, y_train, X_test, y_test)

BayesKeras Warning: HMC is a memory hungry optimizer. 
         Given you system and parameters of this training run,
         we expect your system to need 0.002392 MB of available memory


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
Numerical Integration: 100%|██████████| 20/20 [00:35<00:00,  1.77s/it]


Current  U:  tf.Tensor(4.1266956, shape=(), dtype=float32)
Proposed U:  tf.Tensor(2.627695, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.0066142525, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.01592695, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(4.4357104, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 1, loss: 3.114, acc: 0.647, val_loss: 0.685, val_acc: 0.878
Still in Burn-In state of Markov chain.


Numerical Integration: 100%|██████████| 20/20 [00:35<00:00,  1.77s/it]


Current  U:  tf.Tensor(2.627695, shape=(), dtype=float32)
Proposed U:  tf.Tensor(2.4755416, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.007039245, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.0074577844, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(1.1638517, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 2, loss: 2.497, acc: 0.876, val_loss: 0.486, val_acc: 0.855
Still in Burn-In state of Markov chain.


...

Current  U:  tf.Tensor(5.063541, shape=(), dtype=float32)
Proposed U:  tf.Tensor(5.0690374, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.013841083, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.013827855, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.99453175, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 18, loss: 5.052, acc: 0.922, val_loss: 1.142, val_acc: 0.915
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 25/25 [00:43<00:00,  1.76s/it]


Current  U:  tf.Tensor(5.0690374, shape=(), dtype=float32)
Proposed U:  tf.Tensor(5.1299396, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.016644984, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.016706318, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.94085765, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 19, loss: 5.086, acc: 0.922, val_loss: 1.087, val_acc: 0.916
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 25/25 [00:43<00:00,  1.76s/it]


Current  U:  tf.Tensor(5.1299396, shape=(), dtype=float32)
Proposed U:  tf.Tensor(5.245542, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.015776107, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.015926972, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.89069486, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 20, loss: 5.180, acc: 0.924, val_loss: 1.129, val_acc: 0.918
Out of Burn-In state. Generating samples from the chain


In [ ]:
# Save your approxiate Bayesian posterior
bayes_model.save(experiment + "%s_FCN_Posterior_%s"%(optim, rob))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
